In [1]:
import cobra

In [2]:
import pandas as pd
from cobra.io import load_model

In [3]:
model = load_model("iJO1366") #no need to import from matlab

In [4]:
#KOlist = ('GLCabcpp', 'GLCptspp', 'HEX1', 'PGI', 'PFK', 'FBA', 'TPI', 'GAPD', 'PGK', 'PGM', 'ENO', 'PYK', 'LDH_D', 'PFL', 'ALCD2x', 'PTAr', 'ACKr', 'G6PDH2r', 'PGL', 'GND', 'RPI', 'RPE', 'TKT1', 'TALA', 'TKT2', 'FUM', 'FRD2', 'SUCOAS', 'AKGDH', 'ACONTa', 'ACONTb', 'ICDHyr', 'CS', 'MDH',  'MDH2', 'MDH3', 'ACALD')

In [5]:
KOlist = ["ACALDtex", "DHAtpp", "ETOHtrpp", "EX_ac_e", "EX_hxa_e", "EX_o2_e", "F6PA", "FACOAE60", "FE2t3pp", "FE3tex", "FORtppi", "HEXt2rpp", "PItex", "PPA", "PYRt2rpp"]

In [6]:
len(KOlist)

15

### WT

In [7]:
biomass = "BIOMASS_Ec_iJO1366_core_53p95M"

In [8]:
model.objective = biomass

In [9]:
solution = model.optimize()
print(solution)

<Solution 0.982 at 0x7fbcc38986d0>


In [10]:
model.reactions.get_by_id("EX_succ_e").summary().to_frame().loc["EX_succ_e", "flux"]

0.0

### MT

In [11]:
model.objective = 'EX_succ_e'

In [12]:
print(model.objective.expression)
print(model.objective.direction)

1.0*EX_succ_e - 1.0*EX_succ_e_reverse_a9039
max


In [13]:
solution = model.optimize()
print(solution)

<Solution 17.096 at 0x7fbcab2173a0>


### MT 0.5 Growth rate

In [14]:
model.reactions.get_by_id(biomass).lower_bound = 0.98*0.5

In [15]:
solution = model.optimize()
print(solution)

<Solution 8.582 at 0x7fbcde7f1190>


In [16]:
from itertools import combinations

In [17]:
#KOlist = ['GAPD', 'PGK', 'PGM', 'ENO', 'TPI', 'ACONTb', 'ACONTa', 'CS', 'FUM', 'MDH', 'GLCptspp', 'PGI']

targetNum = 12
targetList = KOlist[:targetNum]
combList = []
for i in range(1,targetNum+1):
    combList.extend(list(combinations(targetList, i)))

In [18]:
len(combList)

4095

In [19]:
model.objective = 'EX_succ_e'

In [20]:
objList = []
for comb in combList:
    with model:
        for rec in comb:
            model.reactions.get_by_id(rec).knock_out()
        objList.append(model.optimize().objective_value)
        #print(model.optimize().objective_value)

/home/chenjunyu/miniconda3/envs/cobra/lib/python3.8/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
/home/chenjunyu/miniconda3/envs/cobra/lib/python3.8/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
/home/chenjunyu/miniconda3/envs/cobra/lib/python3.8/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
/home/chenjunyu/miniconda3/envs/cobra/lib/python3.8/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
/home/chenjunyu/miniconda3/envs/cobra/lib/python3.8/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
/home/chenjunyu/miniconda3/envs/cobra/lib/python3.8/sit

In [21]:
cList = []
for comb in combList:
    cList.append(comb[0])

In [22]:
df = pd.DataFrame()
df["comb"] = combList
df["obj"] = objList

In [23]:
df

,comb,obj
0,"(ACALDtex,)",8.581712
1,"(DHAtpp,)",8.581712
2,"(ETOHtrpp,)",8.581712
3,"(EX_ac_e,)",8.581712
4,"(EX_hxa_e,)",8.581712
...,...,...
4090,"(ACALDtex, DHAtpp, ETOHtrpp, EX_hxa_e, EX_o2_e...",0.000000
4091,"(ACALDtex, DHAtpp, EX_ac_e, EX_hxa_e, EX_o2_e,...",0.000000
4092,"(ACALDtex, ETOHtrpp, EX_ac_e, EX_hxa_e, EX_o2_...",0.000000
4093,"(DHAtpp, ETOHtrpp, EX_ac_e, EX_hxa_e, EX_o2_e,...",0.000000


In [24]:
df.to_csv("Ecoli_Suc_KO_12.csv")

In [52]:
import numpy as np

In [102]:
for ko in KOlist[:10]:
    df[ko] = 0

In [103]:
df

,comb,obj,GAPD,PGK,PGM,ENO,TPI,ACONTb,ACONTa,CS,FUM,MDH
0,"(GAPD,)",14.151613,0,0,0,0,0,0,0,0,0,0
1,"(PGK,)",14.151613,0,0,0,0,0,0,0,0,0,0
2,"(PGM,)",14.796774,0,0,0,0,0,0,0,0,0,0
3,"(ENO,)",14.796774,0,0,0,0,0,0,0,0,0,0
4,"(TPI,)",16.087097,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1018,"(GAPD, PGK, PGM, TPI, ACONTb, ACONTa, CS, FUM,...",1.264935,0,0,0,0,0,0,0,0,0,0
1019,"(GAPD, PGK, ENO, TPI, ACONTb, ACONTa, CS, FUM,...",1.264935,0,0,0,0,0,0,0,0,0,0
1020,"(GAPD, PGM, ENO, TPI, ACONTb, ACONTa, CS, FUM,...",1.264935,0,0,0,0,0,0,0,0,0,0
1021,"(PGK, PGM, ENO, TPI, ACONTb, ACONTa, CS, FUM, ...",1.264935,0,0,0,0,0,0,0,0,0,0


In [57]:
type(df["comb"][0])

tuple

In [58]:
df["comb"][0]

('GAPD',)

In [62]:
for comb in df["comb"][0]:
    print(comb)

GAPD


In [89]:
df.loc[0, "GAPD"] = 1

In [90]:
df["GAPD"][0]

1

In [72]:
len(df)

1023

In [74]:
df["comb"][10]

('GAPD', 'PGK')

In [104]:
for i in range(len(df)):
    #print(i)
    for j in df["comb"][i]:
        df.loc[i, j] = 1

In [105]:
df

,comb,obj,GAPD,PGK,PGM,ENO,TPI,ACONTb,ACONTa,CS,FUM,MDH
0,"(GAPD,)",14.151613,1,0,0,0,0,0,0,0,0,0
1,"(PGK,)",14.151613,0,1,0,0,0,0,0,0,0,0
2,"(PGM,)",14.796774,0,0,1,0,0,0,0,0,0,0
3,"(ENO,)",14.796774,0,0,0,1,0,0,0,0,0,0
4,"(TPI,)",16.087097,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1018,"(GAPD, PGK, PGM, TPI, ACONTb, ACONTa, CS, FUM,...",1.264935,1,1,1,0,1,1,1,1,1,1
1019,"(GAPD, PGK, ENO, TPI, ACONTb, ACONTa, CS, FUM,...",1.264935,1,1,0,1,1,1,1,1,1,1
1020,"(GAPD, PGM, ENO, TPI, ACONTb, ACONTa, CS, FUM,...",1.264935,1,0,1,1,1,1,1,1,1,1
1021,"(PGK, PGM, ENO, TPI, ACONTb, ACONTa, CS, FUM, ...",1.264935,0,1,1,1,1,1,1,1,1,1


In [110]:
df.to_csv("Ecoli_Suc_KO_10_PCA.csv")